In [ ]:
# Configuración del entorno
import sys
from pathlib import Path

# Agregar el directorio raíz del proyecto al path para importar módulos
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Importar configuración y módulos del proyecto
from config import (
    DATA_RAW_CSV_DIR,
    DATA_PROCESSED_DIR,
    OUTPUT_FIGURES_DIR,
    AÑOS_RANGO,
    TIPOS_DATOS
)
from scripts.data_cleaning import (
    cargar_tipo_normalizado,
    consolidar_todos_tipos,
    resumen_carga,
    validar_dataset,
    guardar_dataset
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("✓ Entorno configurado correctamente")
print(f"✓ Directorio de datos: {DATA_RAW_CSV_DIR}")
print(f"✓ Años a procesar: {AÑOS_RANGO[0]}-{AÑOS_RANGO[1]-1}")
print(f"✓ Tipos de datos: {TIPOS_DATOS}")

## ANÁLISIS EXPLORATORIO CC3074 SECCIÓN 20 
#### DIEGO LÓPEZ - 23747

## 1.2 - Exploración de Archivos 2009 (Muestra Representativa)

In [ ]:
# Archivos de 2009 (usando configuración centralizada)
files_2009 = sorted(DATA_RAW_CSV_DIR.glob("2009_*.csv"))
print(f"Archivos de 2009 encontrados: {len(files_2009)}\n")

# Cargar todos los archivos de 2009
dfs_2009 = {}
for file in files_2009:
    name = file.stem.replace("2009_", "")
    dfs_2009[name] = pd.read_csv(file)
    print(f" {name}: {dfs_2009[name].shape}")

print(f"\nTotal archivos cargados: {len(dfs_2009)}")

Archivos de 2009 encontrados: 5

 defunciones: (96001, 27)
 defunciones_fetales: (2306, 29)
 divorcios: (4074, 19)
 matrimonios: (57387, 22)
 nacimientos: (341212, 42)

Total archivos cargados: 5


In [14]:
# Comparar estructura de columnas
print("COLUMNAS POR ARCHIVO 2009:\n")
for name, df in dfs_2009.items():
    print(f"{name}:")
    print(f"  Columnas: {list(df.columns)}")
    print(f"  Tipos: {df.dtypes.to_dict()}")
    print()

COLUMNAS POR ARCHIVO 2009:

defunciones:
  Columnas: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu', 'Mupocu', 'Sexo', 'Diaocu', 'Mesocu', 'Añoocu', 'Edadif', 'Perdif', 'Puedif', 'Ecidif', 'Escodif', 'Ciuodif', 'Pnadif', 'Dnadif', 'Mnadif', 'Nacdif', 'Predif', 'Dredif', 'Mredif', 'Caudef', 'Asist', 'Ocur', 'Cerdef']
  Tipos: {'Depreg': dtype('float64'), 'Mupreg': dtype('int64'), 'Mesreg': dtype('float64'), 'Añoreg': dtype('float64'), 'Depocu': dtype('float64'), 'Mupocu': dtype('int64'), 'Sexo': dtype('float64'), 'Diaocu': dtype('float64'), 'Mesocu': dtype('float64'), 'Añoocu': dtype('float64'), 'Edadif': dtype('float64'), 'Perdif': dtype('float64'), 'Puedif': dtype('float64'), 'Ecidif': dtype('float64'), 'Escodif': dtype('float64'), 'Ciuodif': dtype('int64'), 'Pnadif': dtype('float64'), 'Dnadif': dtype('float64'), 'Mnadif': dtype('int64'), 'Nacdif': dtype('float64'), 'Predif': dtype('float64'), 'Dredif': dtype('float64'), 'Mredif': dtype('int64'), 'Caudef': dtype('O'), 'Asist': dtyp

In [15]:
# Ver primeras filas de cada archivo
print("PRIMERAS FILAS POR ARCHIVO 2009:\n")
for name, df in dfs_2009.items():
    print(f"{name}:")
    display(df.head(3))
    print()

PRIMERAS FILAS POR ARCHIVO 2009:

defunciones:


,Depreg,Mupreg,Mesreg,Añoreg,Depocu,Mupocu,Sexo,Diaocu,Mesocu,Añoocu,...,Dnadif,Mnadif,Nacdif,Predif,Dredif,Mredif,Caudef,Asist,Ocur,Cerdef
0,1.0,101,4.0,2020.0,1.0,101,1.0,18.0,4.0,2020.0,...,1.0,101,320.0,320.0,1.0,101,I776,1.0,1.0,1.0
1,1.0,101,7.0,2020.0,1.0,101,1.0,7.0,7.0,2020.0,...,1.0,101,320.0,320.0,1.0,101,U071,1.0,1.0,9.0
2,1.0,101,2.0,2020.0,1.0,101,2.0,6.0,2.0,2020.0,...,1.0,101,320.0,320.0,1.0,101,R99X,1.0,4.0,1.0



defunciones_fetales:


,DEPREG,MUPREG,MESREG,AÑOREG,DEPOCU,MUPOCU,SEXO,DIAOCU,MESOCU,AÑOOCU,...,ESCIVM,NACIONM,ESCOLAM,CIUOMAD,CAUDEF,ASISREC,SITIOOCU,TOHITE,TOHINM,TOHIVI
0,1.0,101,12.0,2020.0,1.0,101,1.0,6.0,12.0,2020.0,...,1.0,320.0,1.0,51,P209,3.0,6.0,2.0,1.0,1.0
1,14.0,1406,2.0,2021.0,14.0,1406,1.0,26.0,11.0,2020.0,...,1.0,320.0,1.0,73,P95X,3.0,6.0,4.0,1.0,3.0
2,16.0,1601,12.0,2020.0,16.0,1601,1.0,30.0,11.0,2020.0,...,1.0,320.0,1.0,73,P95X,1.0,1.0,99.0,99.0,99.0



divorcios:


,DEPREG,MUPREG,MESREG,AÑOREG,DIAOCU,MESOCU,AÑOOCU,DEPOCU,MUPOCU,EDADHOM,EDADMUJ,PPERHOM,PPERMUJ,NACHOM,NACMUJ,ESCHOM,ESCMUJ,CIUOHOM,CIUOMUJ
0,6.0,606,12.0,2020.0,19.0,11.0,2020.0,6.0,614,19.0,23.0,4.0,4.0,320.0,320.0,2.0,2.0,92.0,97.0
1,7.0,704,2.0,2020.0,3.0,2.0,2020.0,7.0,704,20.0,20.0,1.0,9.0,320.0,320.0,4.0,4.0,72.0,34.0
2,9.0,916,9.0,2020.0,21.0,9.0,2020.0,9.0,916,20.0,20.0,1.0,1.0,320.0,320.0,4.0,3.0,74.0,97.0



matrimonios:


,DEPREG,MUPREG,MESREG,AÑOREG,AÑOOCU,CLAUNI,NUNUHO,NUNUMU,EDADHOM,EDADMUJ,...,NACHOM,NACMUJ,ESCHOM,ESCMUJ,CIUOHOM,CIUOMUJ,DEPOCU,MUPOCU,DIAOCU,MESOCU
0,1.0,101,2.0,2021.0,2020.0,3.0,9.0,9.0,33.0,33.0,...,320.0,320.0,4.0,4.0,1.0,1.0,1.0,115,20.0,12.0
1,1.0,101,4.0,2021.0,2020.0,3.0,9.0,9.0,30.0,31.0,...,320.0,320.0,4.0,4.0,1.0,1.0,1.0,116,20.0,12.0
2,1.0,101,3.0,2021.0,2020.0,2.0,9.0,9.0,37.0,34.0,...,320.0,862.0,1.0,9.0,97.0,11.0,1.0,101,26.0,9.0



nacimientos:


,Depreg,Mupreg,Mesreg,Añoreg,TipoIns,Depocu,Mupocu,Libras,Onzas,Diaocu,...,Paisnacm,Depnam,Mupnam,Escolam,Ocupam,Asisrec,Sitioocu,Tohite,Tohinm,Tohivi
0,2.0,207,1.0,2021.0,3.0,2.0,207,5.0,12.0,22.0,...,32.0,23.0,2300,1.0,97.0,1.0,1.0,1.0,0.0,1.0
1,13.0,1311,1.0,2020.0,3.0,13.0,1306,6.0,12.0,13.0,...,32.0,23.0,2300,2.0,97.0,1.0,1.0,4.0,0.0,4.0
2,1.0,101,5.0,2020.0,3.0,1.0,101,6.0,6.0,28.0,...,32.0,23.0,2300,1.0,97.0,1.0,1.0,2.0,0.0,2.0


In [16]:
# Análisis de valores faltantes en 2009
print("VALORES FALTANTES EN 2009:\n")
for name, df in dfs_2009.items():
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"{name}:")
        print(missing[missing > 0])
    else:
        print(f"{name}: Sin valores faltantes")
    print()

VALORES FALTANTES EN 2009:

defunciones: Sin valores faltantes

defunciones_fetales: Sin valores faltantes

divorcios: Sin valores faltantes

matrimonios: Sin valores faltantes

nacimientos: Sin valores faltantes



### Normalización - DataSet Maestro

In [20]:
# Ver columnas de CADA TIPO (no todas iguales)
for nombre, df in dfs_2009.items():
    print(f"\n{nombre.upper()}:")
    print(f"  Total columnas: {len(df.columns)}")
    print(f"  Lista: {list(df.columns)}")


DEFUNCIONES:
  Total columnas: 27
  Lista: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu', 'Mupocu', 'Sexo', 'Diaocu', 'Mesocu', 'Añoocu', 'Edadif', 'Perdif', 'Puedif', 'Ecidif', 'Escodif', 'Ciuodif', 'Pnadif', 'Dnadif', 'Mnadif', 'Nacdif', 'Predif', 'Dredif', 'Mredif', 'Caudef', 'Asist', 'Ocur', 'Cerdef']

DEFUNCIONES_FETALES:
  Total columnas: 29
  Lista: ['DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'DEPOCU', 'MUPOCU', 'SEXO', 'DIAOCU', 'MESOCU', 'AÑOOCU', 'TIPAR', 'CLAPAR', 'VIAPAR', 'SEMGES', 'EDADM', 'PAISREM', 'DEPREM', 'MUPREM', 'PUEBLOPM', 'ESCIVM', 'NACIONM', 'ESCOLAM', 'CIUOMAD', 'CAUDEF', 'ASISREC', 'SITIOOCU', 'TOHITE', 'TOHINM', 'TOHIVI']

DIVORCIOS:
  Total columnas: 19
  Lista: ['DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'DIAOCU', 'MESOCU', 'AÑOOCU', 'DEPOCU', 'MUPOCU', 'EDADHOM', 'EDADMUJ', 'PPERHOM', 'PPERMUJ', 'NACHOM', 'NACMUJ', 'ESCHOM', 'ESCMUJ', 'CIUOHOM', 'CIUOMUJ']

MATRIMONIOS:
  Total columnas: 22
  Lista: ['DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'AÑOOCU', 'CLAUN

In [31]:
# PASO 1: Cargar DEFUNCIONES (14 años: 2009-2022)

lista_dfs = []  # Lista temporal para almacenar cada año

# Definir rango de años (2009-2022 = 14 años)
for año in range(2009, 2023):
    archivo = csv_dir / f"{año}_defunciones.csv"
    
    # Cargar el CSV
    df = pd.read_csv(archivo)
    
    # Agregar columna 'año' 
    df['año'] = año
    
    # Agregar a la lista
    lista_dfs.append(df)
    print(f"✓ {año}: {df.shape[0]:,} filas, {df.shape[1]} columnas")

# Concatenar todos los años verticalmente
defunciones_completo = pd.concat(lista_dfs, ignore_index=True)

print(f"\n{'='*60}")
print(f"DATASET CONSOLIDADO: defunciones")
print(f"{'='*60}")
print(f"Total filas: {defunciones_completo.shape[0]:,}")
print(f"Total columnas: {defunciones_completo.shape[1]}")
print(f"Años: {defunciones_completo['año'].min()} - {defunciones_completo['año'].max()}")
print(f"Memoria: {defunciones_completo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

✓ 2009: 96,001 filas, 28 columnas
✓ 2010: 96,001 filas, 28 columnas
✓ 2011: 96,001 filas, 28 columnas
✓ 2012: 72,657 filas, 28 columnas


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\348815590.py:10: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)
C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\348815590.py:10: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)


✓ 2013: 72,657 filas, 28 columnas
✓ 2014: 77,807 filas, 28 columnas
✓ 2015: 80,876 filas, 29 columnas
✓ 2016: 80,876 filas, 29 columnas
✓ 2017: 3,033 filas, 31 columnas
✓ 2018: 2,894 filas, 30 columnas
✓ 2019: 85,600 filas, 28 columnas
✓ 2020: 96,001 filas, 28 columnas
✓ 2021: 96,001 filas, 28 columnas
✓ 2022: 95,386 filas, 28 columnas

DATASET CONSOLIDADO: defunciones
Total filas: 1,051,791
Total columnas: 61
Años: 2009 - 2022
Memoria: 615.73 MB


In [32]:
# Ver las 61 columnas de defunciones_completo
print("COLUMNAS DE DEFUNCIONES CONSOLIDADO:\n")
print(defunciones_completo.columns.tolist())

print("\n" + "="*60)
print("VALORES NO NULOS POR COLUMNA:")
print("="*60)
info_cols = defunciones_completo.count().sort_values(ascending=False)
for col, count in info_cols.items():
    porcentaje = (count / len(defunciones_completo)) * 100
    print(f"{col:30s}: {count:>10,} ({porcentaje:5.1f}%)")

COLUMNAS DE DEFUNCIONES CONSOLIDADO:

['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu', 'Mupocu', 'Sexo', 'Diaocu', 'Mesocu', 'Añoocu', 'Edadif', 'Perdif', 'Puedif', 'Ecidif', 'Escodif', 'Ciuodif', 'Pnadif', 'Dnadif', 'Mnadif', 'Nacdif', 'Predif', 'Dredif', 'Mredif', 'Caudef', 'Asist', 'Ocur', 'Cerdef', 'año', 'Areag', 'Getdif', 'Ocudif', 'DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'DEPOCU', 'MUPOCU', 'AREAG', 'SEXO', 'DIAOCU', 'MESOCU', 'AÑOOCU', 'TIPAR', 'CLAPAR', 'VIAPAR', 'SEMGES', 'EDADM', 'PAISREM', 'DEPREM', 'MUPREM', 'PUEBLOPM', 'ESCIVM', 'NACIONM', 'ESCOLAM', 'CIUOMAD', 'CAUDEF', 'ASISREC', 'SITIOOCU', 'TOHITE', 'TOHINM', 'TOHIVI']

VALORES NO NULOS POR COLUMNA:
año                           :  1,051,791 (100.0%)
Mupreg                        :  1,045,864 ( 99.4%)
Mesreg                        :  1,045,864 ( 99.4%)
Añoreg                        :  1,045,864 ( 99.4%)
Depreg                        :  1,045,864 ( 99.4%)
Mupocu                        :  1,045,864 ( 99.4%)
Sexo       

## FASE 1: INVESTIGACIÓN - Detectar cambios de capitalización por año

In [34]:
# INVESTIGACIÓN: ¿QUÉ AÑO CAMBIA DE CAPITALIZACIÓN?

print("="*70)
print("COLUMNAS POR AÑO - DETECCIÓN DE CAMBIOS DE FORMATO")
print("="*70)

for año in range(2009, 2023):
    archivo = csv_dir / f"{año}_defunciones.csv"
    df = pd.read_csv(archivo)
    cols = df.columns.tolist()
    
    # Detectar patrón: MixedCase (Depreg) vs UPPERCASE (DEPREG)
    # MixedCase = primera letra mayúscula, resto minúsculas (o mezcla)
    # UPPERCASE = todas las letras mayúsculas
    
    uppercase_cols = [c for c in cols if c == c.upper() and c != c.lower()]  # Todo mayusculas (DEPREG, AREAG)
    mixed_case_cols = [c for c in cols if c != c.upper() and c != c.lower()]  # MixedCase (Depreg, Sexo)
    
    print(f"\n{año}:")
    print(f"   Total columnas: {len(cols)}")
    print(f"   UPPERCASE (DEPREG style): {len(uppercase_cols)}")
    if uppercase_cols:
        print(f"      Ejemplos: {uppercase_cols[:5]}")
    print(f"   MixedCase (Depreg style): {len(mixed_case_cols)}")
    if mixed_case_cols:
        print(f"      Ejemplos: {mixed_case_cols[:5]}")
    
    if len(uppercase_cols) > 0:
        print(f"   >>> CAMBIO DETECTADO: Ahora hay columnas UPPERCASE")
    else:
        print(f"   --> Mantiene formato MixedCase")

COLUMNAS POR AÑO - DETECCIÓN DE CAMBIOS DE FORMATO

2009:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2010:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2011:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\2014599505.py:9: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)



2012:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\2014599505.py:9: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)



2013:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2014:
   Total columnas: 27
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 27
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2015:
   Total columnas: 28
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 28
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2016:
   Total columnas: 28
   UPPERCASE (DEPREG style): 0
   MixedCase (Depreg style): 28
      Ejemplos: ['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu']
   --> Mantiene formato MixedCase

2017:
   Total columnas: 30
   UPPERCASE (DEPREG style): 30
      Ejemplos: ['DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'DEPOCU']
   MixedCase (Depreg style): 0
   >>> CAMBIO DETECTADO: Ahora hay columnas UPPERCASE

2018:
   Total

In [35]:
# RESUMEN CONSOLIDADO: Recopilar tipos de columnas por año

datos_años = {}  # Dict para almacenar info por año

for año in range(2009, 2023):
    archivo = csv_dir / f"{año}_defunciones.csv"
    df = pd.read_csv(archivo)
    cols = df.columns.tolist()
    
    # Clasificar columnas
    uppercase_cols = set([c for c in cols if c == c.upper() and c != c.lower()])
    mixed_case_cols = set([c for c in cols if c != c.upper() and c != c.lower()])
    
    datos_años[año] = {
        'total_cols': len(cols),
        'uppercase': uppercase_cols,
        'mixedcase': mixed_case_cols
    }

print("="*70)
print("RESUMEN: CAMBIOS DE FORMATO POR AÑO")
print("="*70)

primer_uppercase = None
último_mixedcase = None

for año in sorted(datos_años.keys()):
    info = datos_años[año]
    if info['uppercase']:
        if primer_uppercase is None:
            primer_uppercase = año
        print(f"\n{año}: TIENE COLUMNAS UPPERCASE")
        print(f"   Columnas UPPERCASE: {sorted(info['uppercase'])}")
    else:
        último_mixedcase = año
        print(f"\n{año}: Solo MixedCase (total: {info['total_cols']} cols)")

print("\n" + "="*70)
print("PUNTOS CLAVE:")
print("="*70)
print(f"Años SOLO MixedCase: 2009-{último_mixedcase}")
print(f"Primer año con UPPERCASE: {primer_uppercase}")
print(f"\nEsto significa que años {primer_uppercase}-2022 tienen COLUMNAS DUPLICADAS")
print(f"(misma info en dos nombres: 'Depreg' y 'DEPREG')")


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\484351958.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)
C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\484351958.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)


RESUMEN: CAMBIOS DE FORMATO POR AÑO

2009: Solo MixedCase (total: 27 cols)

2010: Solo MixedCase (total: 27 cols)

2011: Solo MixedCase (total: 27 cols)

2012: Solo MixedCase (total: 27 cols)

2013: Solo MixedCase (total: 27 cols)

2014: Solo MixedCase (total: 27 cols)

2015: Solo MixedCase (total: 28 cols)

2016: Solo MixedCase (total: 28 cols)

2017: TIENE COLUMNAS UPPERCASE
   Columnas UPPERCASE: ['AREAG', 'ASISREC', 'AÑOOCU', 'AÑOREG', 'CAUDEF', 'CIUOMAD', 'CLAPAR', 'DEPOCU', 'DEPREG', 'DEPREM', 'DIAOCU', 'EDADM', 'ESCIVM', 'ESCOLAM', 'MESOCU', 'MESREG', 'MUPOCU', 'MUPREG', 'MUPREM', 'NACIONM', 'PAISREM', 'PUEBLOPM', 'SEMGES', 'SEXO', 'SITIOOCU', 'TIPAR', 'TOHINM', 'TOHITE', 'TOHIVI', 'VIAPAR']

2018: TIENE COLUMNAS UPPERCASE
   Columnas UPPERCASE: ['ASISREC', 'AÑOOCU', 'AÑOREG', 'CAUDEF', 'CIUOMAD', 'CLAPAR', 'DEPOCU', 'DEPREG', 'DEPREM', 'DIAOCU', 'EDADM', 'ESCIVM', 'ESCOLAM', 'MESOCU', 'MESREG', 'MUPOCU', 'MUPREG', 'MUPREM', 'NACIONM', 'PAISREM', 'PUEBLOPM', 'SEMGES', 'SEXO', 'S

## FASE 2: VALIDACIÓN - Verificar que columnas duplicadas sean idénticas

In [36]:
# VALIDACIÓN: ¿Las columnas duplicadas tienen datos idénticos?
# Usar año 2017 como prueba (primer año con duplicados)

print("="*70)
print("VALIDACIÓN: Verificar que duplicados sean idénticos")
print("="*70)

df_2017 = pd.read_csv(csv_dir / "2017_defunciones.csv")
cols_2017 = df_2017.columns.tolist()

# Encontrar pares: MixedCase vs UPPERCASE
mapeo_duplicados = {}  # {MixedCase: UPPERCASE}

for col in cols_2017:
    if col != col.upper():  # Si es MixedCase
        col_upper = col.upper()
        if col_upper in cols_2017:
            mapeo_duplicados[col] = col_upper

print(f"\nEncontrados {len(mapeo_duplicados)} pares de duplicados en 2017:\n")

# Verificar que cada par tenga EXACTAMENTE los mismos valores
validación_ok = True
for col_mixed, col_upper in sorted(mapeo_duplicados.items())[:5]:  # Mostrar primeros 5
    # Comparar valores
    son_iguales = (df_2017[col_mixed] == df_2017[col_upper]).all()
    valores_nulos_mezcla = (df_2017[col_mixed].isna() == df_2017[col_upper].isna()).all()
    
    estado = "OK" if (son_iguales and valores_nulos_mezcla) else "DIFERENTE"
    print(f"  {col_mixed:20s} <-> {col_upper:20s} : {estado}")
    
    if not (son_iguales and valores_nulos_mezcla):
        validación_ok = False

print(f"\nValidación general: {'TODAS IDENTICAS' if validación_ok else 'ALGUNAS DIFERENTES'}")
print(f"\nDICCIONARIO DE MAPEO (27 pares):")
print(mapeo_duplicados)


VALIDACIÓN: Verificar que duplicados sean idénticos

Encontrados 0 pares de duplicados en 2017:


Validación general: TODAS IDENTICAS

DICCIONARIO DE MAPEO (27 pares):
{}


## FASE 3: SOLUCIÓN - Cargar normalizando columnas a lowercase

In [37]:
# RECARGAR DEFUNCIONES CON NORMALIZACIÓN A LOWERCASE

print("="*70)
print("RECARGANDO: DEFUNCIONES NORMALIZADOS A LOWERCASE")
print("="*70)

lista_dfs_clean = []

for año in range(2009, 2023):
    archivo = csv_dir / f"{año}_defunciones.csv"
    df = pd.read_csv(archivo)
    
    # PASO CRÍTICO: Normalizar TODOS los nombres de columnas a lowercase
    df.columns = df.columns.str.lower()
    
    # Agregar año
    df['año'] = año
    
    lista_dfs_clean.append(df)
    print(f"✓ {año}: {df.shape[0]:,} filas, {df.shape[1]} columnas")

# Concatenar verticalmente
defunciones_clean = pd.concat(lista_dfs_clean, ignore_index=True)

print(f"\n{'='*60}")
print(f"DATASET LIMPIO: defunciones (normalizado)")
print(f"{'='*60}")
print(f"Total filas: {defunciones_clean.shape[0]:,}")
print(f"Total columnas: {defunciones_clean.shape[1]}")
print(f"Años: {defunciones_clean['año'].min()} - {defunciones_clean['año'].max()}")
print(f"Memoria: {defunciones_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nREDUCCION:")
print(f"  Antes (61 cols) -> Después ({defunciones_clean.shape[1]} cols)")
print(f"  Eliminadas: {61 - defunciones_clean.shape[1]} columnas duplicadas")


RECARGANDO: DEFUNCIONES NORMALIZADOS A LOWERCASE
✓ 2009: 96,001 filas, 28 columnas
✓ 2010: 96,001 filas, 28 columnas
✓ 2011: 96,001 filas, 28 columnas


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\3798667501.py:11: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)


✓ 2012: 72,657 filas, 28 columnas


C:\Users\djlop\AppData\Local\Temp\ipykernel_17828\3798667501.py:11: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)


✓ 2013: 72,657 filas, 28 columnas
✓ 2014: 77,807 filas, 28 columnas
✓ 2015: 80,876 filas, 29 columnas
✓ 2016: 80,876 filas, 29 columnas
✓ 2017: 3,033 filas, 31 columnas
✓ 2018: 2,894 filas, 30 columnas
✓ 2019: 85,600 filas, 28 columnas
✓ 2020: 96,001 filas, 28 columnas
✓ 2021: 96,001 filas, 28 columnas
✓ 2022: 95,386 filas, 28 columnas

DATASET LIMPIO: defunciones (normalizado)
Total filas: 1,051,791
Total columnas: 49
Años: 2009 - 2022
Memoria: 495.36 MB

REDUCCION:
  Antes (61 cols) -> Después (49 cols)
  Eliminadas: 12 columnas duplicadas


In [38]:
# VERIFICACIÓN: Ver columnas limpias y su completitud

print("="*70)
print("COLUMNAS DEL DATASET LIMPIO (normalizado)")
print("="*70)
print(f"\nTotal columnas: {defunciones_clean.shape[1]}")
print(f"\nLista completa de columnas:")
print(defunciones_clean.columns.tolist())

print("\n" + "="*70)
print("COMPLETITUD DE DATOS (por columna)")
print("="*70)

completitud = defunciones_clean.count().sort_values(ascending=False)
for col, count in completitud.items():
    porcentaje = (count / len(defunciones_clean)) * 100
    print(f"{col:30s}: {count:>10,} ({porcentaje:5.1f}%)")


COLUMNAS DEL DATASET LIMPIO (normalizado)

Total columnas: 49

Lista completa de columnas:
['depreg', 'mupreg', 'mesreg', 'añoreg', 'depocu', 'mupocu', 'sexo', 'diaocu', 'mesocu', 'añoocu', 'edadif', 'perdif', 'puedif', 'ecidif', 'escodif', 'ciuodif', 'pnadif', 'dnadif', 'mnadif', 'nacdif', 'predif', 'dredif', 'mredif', 'caudef', 'asist', 'ocur', 'cerdef', 'año', 'areag', 'getdif', 'ocudif', 'tipar', 'clapar', 'viapar', 'semges', 'edadm', 'paisrem', 'deprem', 'muprem', 'pueblopm', 'escivm', 'nacionm', 'escolam', 'ciuomad', 'asisrec', 'sitioocu', 'tohite', 'tohinm', 'tohivi']

COMPLETITUD DE DATOS (por columna)
depreg                        :  1,051,791 (100.0%)
mupreg                        :  1,051,791 (100.0%)
mesreg                        :  1,051,791 (100.0%)
añoreg                        :  1,051,791 (100.0%)
depocu                        :  1,051,791 (100.0%)
mupocu                        :  1,051,791 (100.0%)
sexo                          :  1,051,791 (100.0%)
diaocu            

---

## FASE 4: CONSOLIDACIÓN DE TODOS LOS TIPOS DE DATOS

Ahora que hemos validado el proceso de normalización con `defunciones`, vamos a cargar **todos los 5 tipos** de estadísticas vitales usando el módulo `data_cleaning.py` que creamos.

In [ ]:
# Cargar todos los tipos de datos (2009-2022, 14 años)
print("🔄 Cargando todos los tipos de datos...\n")

dfs_por_tipo, master_dataset = consolidar_todos_tipos(
    tipos_lista=TIPOS_DATOS,
    años_rango=AÑOS_RANGO,
    csv_dir=DATA_RAW_CSV_DIR,
    verbose=True
)

print("\n" + "="*80)
print("✅ CARGA COMPLETADA")
print("="*80)

In [ ]:
# Resumen de cada tipo de dato
print("\n📊 RESUMEN INDIVIDUAL POR TIPO\n")
print("="*80)

for tipo in TIPOS_DATOS:
    if tipo in dfs_por_tipo:
        df = dfs_por_tipo[tipo]
        print(f"\n{tipo.upper()}:")
        print(f"  • Filas: {len(df):,}")
        print(f"  • Columnas: {len(df.columns)}")
        print(f"  • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        print(f"  • Rango años: {df['año'].min()} - {df['año'].max()}")
        print(f"  • Nulos: {df.isnull().sum().sum():,} valores ({df.isnull().sum().sum() / df.size * 100:.2f}%)")
        
print("\n" + "="*80)

In [ ]:
# Resumen del dataset maestro
print("\n🎯 DATASET MAESTRO CONSOLIDADO\n")
print("="*80)
print(f"  • Total de registros: {len(master_dataset):,}")
print(f"  • Total de columnas: {len(master_dataset.columns)}")
print(f"  • Memoria total: {master_dataset.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  • Tipos incluidos: {master_dataset['tipo'].unique().tolist()}")
print(f"  • Rango temporal: {master_dataset['año'].min()} - {master_dataset['año'].max()}")
print("\n📈 Distribución por tipo:")
print(master_dataset['tipo'].value_counts())
print("="*80)

### Guardar Datos Procesados

Ahora guardamos cada tipo de dato limpio y normalizado en `data/processed/` para análisis posteriores.

In [ ]:
# Guardar datasets individuales
print("💾 Guardando datasets limpios...\n")

for tipo in TIPOS_DATOS:
    if tipo in dfs_por_tipo:
        ruta = DATA_PROCESSED_DIR / f"{tipo}_clean.csv"
        guardar_dataset(dfs_por_tipo[tipo], ruta, f"{tipo} limpio")

# Guardar dataset maestro
ruta_master = DATA_PROCESSED_DIR / "master_dataset.csv"
guardar_dataset(master_dataset, ruta_master, "Dataset maestro")

print("\n✅ Todos los archivos guardados en data/processed/")